In [1]:
from scipy import io
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline

# Simulate data

In [ ]:
dm = io.loadmat('GazeCenterFS_vd.mat')['stim']

# braincoder convention is time by x by y
paradigm = np.rollaxis(dm, 2, 0)
paradigm.shape

In [2]:
import prfpy_csenf
import os 
from os.path import join as opj
seq_path = os.path.join(os.path.dirname(prfpy_csenf.__path__[0]))
sfs_seq = np.load(opj(seq_path, 'eg_sfs_seq.npy'))
con_seq = np.load(opj(seq_path, 'eg_con_seq.npy'))
paradigm = np.vstack([sfs_seq, con_seq])


In [ ]:
!pip install tensorflow
# Stimulus object 
from braincoder.stimuli import ContrastSensitivityStimulus
cs_stim = ContrastSensitivityStimulus()



In [3]:
parameters = pd.DataFrame({
    #                                
    'width_r'           : np.array([   1.28,   1.50,   1.00,   1.00,]), # [CSF parameters]    
    'SFp'               : np.array([   2.50,   1.00,   4.00,   2.00,]), # 
    'CSp'               : np.array([ 166.00, 100.00,  80.00,  40.00,]), # 
    'width_l'           : np.array([   0.68,   0.68,   0.68,   0.68,]), # 
    'crf_exp'           : np.array([   1.50,   1.50,   2.50,   3.00,]), # [CRF parameter]
    'amplitude'         : np.array([   1.00,   1.00,   1.00,   1.00,]), # [fMRI parameters]
    'baseline'          : np.array([   0.00,   0.00,   0.00,   0.00,]), # 
    # 'hrf_1'             : np.array([   1.00,   1.00,   1.00,   1.00,]), # 
    # 'hrf_2'             : np.array([   0.00,   0.00,   0.00,   0.00,]), # 
    })

In [ ]:
from braincoder.models import ContrastSensitivity
from braincoder.hrf import SPMHRFModel

In [ ]:
model = ContrastSensitivity(
    paradigm=paradigm,
    parameters=parameters,

)

In [4]:
parameters = pd.DataFrame({'x':x.ravel(),
               'y':y.ravel(),
               'sd':.2,
               'amplitude':0.001,
               'baseline':0.0}).astype(np.float32)

y_grid, x_grid = np.meshgrid(np.linspace(-aspect_ratio, aspect_ratio, paradigm.shape[2]), 
                              np.linspace(-1., 1., paradigm.shape[1]))

grid_coordinates = np.stack((x_grid.ravel().astype(np.float32), y_grid.ravel().astype(np.float32)), 1)

In [5]:
from braincoder.models import GaussianPRF2DWithHRF
from braincoder.hrf import SPMHRFModel

In [6]:
model = GaussianPRF2DWithHRF(grid_coordinates, 
                      paradigm=paradigm,
                     parameters=parameters,
                      hrf_model=SPMHRFModel(tr=1.7))

In [ ]:
for i in range(9):
    plt.subplot(3, 3, i+1)
    plt.imshow(model.get_rf()[i].reshape((240, 135)).T)
    plt.axis("off")

## Predictions (no noise)

In [ ]:
pred = model.predict()
pred.plot(legend=False)

## Noisy predictions

In [ ]:
data = model.simulate(noise=35)
data.plot(legend=False)
sns.despine()

# Estimate parameters

In [10]:
from braincoder.optimize import ParameterFitter

In [11]:
fitter = ParameterFitter(model, data, paradigm)

In [ ]:
grid_pars = fitter.fit_grid(x=np.linspace(-1, 1, 20), 
                            y=np.linspace(-aspect_ratio, aspect_ratio, 20),
                            sd=np.linspace(0.01, 1., 10),
                            baseline=[0.0],
                            amplitude=[1.0],
                            use_correlation_cost=True)

In [ ]:
grid_pars

In [ ]:
ols_pars = fitter.refine_baseline_and_amplitude(grid_pars)

In [ ]:
ols_pars

In [ ]:
refined_pars = fitter.fit(init_pars=ols_pars, learning_rate=0.0001)

In [ ]:
refined_pars

In [ ]:
r2 = fitter.get_rsq(refined_pars)
r2

# Mexican hat model

In [19]:
from braincoder.models import DifferenceOfGaussiansPRF2DWithHRF

In [20]:
parameters_dog = pd.DataFrame({'x':x.ravel(),
               'y':y.ravel(),
               'sd':.2,
               'amplitude':0.001,
               'baseline':0.0,
               'srf_amplitude':.6, 
               'srf_factor':2.}).astype(np.float32)

In [21]:
model_dog = DifferenceOfGaussiansPRF2DWithHRF(grid_coordinates, paradigm, parameters=parameters_dog,
                                             hrf_model=SPMHRFModel(tr=1.8))

In [ ]:
pred_dog = model_dog.predict()

In [ ]:
pred[0].plot()
pred_dog[0].plot()

In [ ]:
data_dog = model.simulate(noise=35)
data_dog.plot(legend=False)
sns.despine()

In [25]:
fitter = ParameterFitter(model_dog, data_dog, paradigm)

In [26]:
est_pars_dog = refined_pars.copy()
est_pars_dog['srf_amplitude'] = 0.1
est_pars_dog['srf_factor'] = 2.

In [ ]:
est_pars_dog = fitter.fit(init_pars=est_pars_dog, learning_rate=0.001)

In [ ]:
est_pred = model.predict(parameters=est_pars_dog)

In [ ]:
est_pred[1].plot()
pred_dog[1].plot()
data_dog[1].plot()

In [ ]:
est_pars_dog

In [ ]:
parameters_dog

In [ ]:
r2 = fitter.get_rsq()

In [ ]:
r2